In [65]:
import requests
import pandas as pd

In [146]:
class WorldBankDataPopulation:
    def __init__(self):
        self.base_url = "http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL"

    def fetch_data(self, year, num_countries):
        per_page=1000 # number of records to fetch per page
        page = 1 # initialize the page number
        data_accumulated = pd.DataFrame() # initialize an empty DataFrame

        while len(data_accumulated) < num_countries: # fetch data until we have enough countries
            url = f"{self.base_url}?date={year}&per_page={per_page}&format=json&page={page}" # construct the URL
            response = requests.get(url) # make the HTTP request
            if response.status_code != 200:
                print(f'Error: {response.status_code}')
                return pd.DataFrame()  # return an empty DataFrame in case of an HTTP error

            data = response.json()
            if not data[1] :  # no more data available
                break

            batch_data = pd.DataFrame(data[1]) # convert the data to a DataFrame
            batch_data = batch_data[['country', 'value']] # extract the relevant columns
            batch_data['Country'] = batch_data['country'].apply(lambda x: x['value']) # extract the country name
            batch_data['ID'] = batch_data['country'].apply(lambda x: x['id']) # extract the country ID
            batch_data['Population'] = batch_data['value'] # extract the population value
            batch_data = batch_data[['Country', 'ID', 'Population']].dropna(subset=['Population']) # drop rows with missing population data
            batch_data['Population'] = batch_data['Population'].astype(int) # convert population to integer
            batch_data['Year'] = year # add the year column
            data_accumulated = pd.concat([data_accumulated, batch_data]) # append the batch data to the accumulated data

            page += 1

        return data_accumulated

    def top_countries_by_population(self, num_countries=10, year=2020):
        data_accumulated = self.fetch_data(year, num_countries) # fetch the data
        if data_accumulated.empty:
            return pd.DataFrame()

        full_data = data_accumulated.sort_values(by='Population', ascending=False) # sort the data by population
        full_data = full_data.head(num_countries) # select the top N countries
        full_data.reset_index(drop=True, inplace=True) # reset the index
        return full_data

,Country,ID,Population,Year
0,World,1W,7820205606,2020
1,IDA & IBRD total,ZT,6627317499,2020
2,Low & middle income,XO,6549996251,2020
3,Middle income,XP,5882942581,2020
4,IBRD only,XF,4867091692,2020


In [ ]:
# Example usage:
wb_data = WorldBankDataPopulation()
result = wb_data.top_countries_by_population(num_countries=5, year=2020)
result

In [140]:
import requests

class Currency:
    def __init__(self):
        # Assuming your API key is stored in 'api.key'
        with open('api.key', 'r') as file:
            self.api_key = file.readline().strip()
        self.url = f'http://api.exchangeratesapi.io/v1/latest?access_key={self.api_key}'

    def fetch_rates(self):
        '''
        Fetches the latest currency exchange rates from the API.

        Returns:
        - A dictionary containing the currency exchange rates.
        '''
        response = requests.get(self.url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f'Error: {response.status_code}')
            return None

    def convert_to_currency(self, currency_code):
        '''
        Converts all currencies to the specified currency.

        Args:
        - currency_code: The currency code to convert to (e.g., 'USD', 'EUR', 'MXN', etc.)

        Returns:
        - A dictionary containing the converted currency exchange rates.
        '''
        data = self.fetch_rates()
        if data:
            rates = data['rates']
            target_rate = rates.get(currency_code, None)  # Get the rate for the target currency

            if target_rate is None:
                print(f"Rate for {currency_code} not found.")
                return None

            # Convert all currencies to the specified currency
            converted_rates = {currency: rate / target_rate for currency, rate in rates.items()}
            # format the output similar to the original API structure
            data['rates'] = converted_rates
            data['base'] = currency_code  # update the base currency to the specified currency
            return data
        else:
            print("Failed to fetch rates.")
            return None
        
    def one_cent_of_every_person(self, dataframe, currency_code):
        '''
        Calculates how much money you will have if every person in the population gives you 1 cent in their currency and you convert it to Mexican Pesos (MXN).

        Args:
        - dataframe: A pandas DataFrame containing the population data and currency information.

        Returns:
        - The total population in Mexican Pesos (MXN).
        '''
        rates_data = self.convert_to_currency(currency_code)
        if rates_data is None:
            return f"Failed to convert currencies to {currency_code}."
        
        total_base_currency = 0
        rates = rates_data['rates']
        for index, row in dataframe.iterrows():
            currency = row['Currency']
            population = row['Population']
            rate_to_base_currency = rates.get(currency, 0)
            total_currency = 0.01 * population
            total_base_currency += total_currency / rate_to_base_currency

        return total_base_currency


In [149]:
currency_converter = Currency()
# Convert and fetch all currencies relative to USD
converted_rates = currency_converter.convert_to_currency('AFN')
converted_rates

{'success': True,
 'timestamp': 1715729824,
 'base': 'AFN',
 'date': '2024-05-14',
 'rates': {'AED': 0.050800091381652604,
  'AFN': 1.0,
  'ALL': 1.291432484131453,
  'AMD': 5.361986447356668,
  'ANG': 0.02492355929874492,
  'AOA': 11.590045857707157,
  'ARS': 12.243596056010839,
  'AUD': 0.020884480023243113,
  'AWG': 0.024895080857609008,
  'AZN': 0.023481708739716445,
  'BAM': 0.025051872131415666,
  'BBD': 0.027922376234990234,
  'BDT': 1.6180350885852794,
  'BGN': 0.025005593066094216,
  'BHD': 0.0052117849083571565,
  'BIF': 39.76297701628674,
  'BMD': 0.013830599055582255,
  'BND': 0.018713980350642884,
  'BOB': 0.09555527237672412,
  'BRL': 0.07093993753541024,
  'BSD': 0.013828616945848953,
  'BTC': 2.2452803249841544e-07,
  'BTN': 1.1549214409988913,
  'BWP': 0.1880235686913875,
  'BYN': 0.04525682099905875,
  'BYR': 271.07977377861914,
  'BZD': 0.027874984630657185,
  'CAD': 0.018884850997458357,
  'CDF': 38.72567803338397,
  'CHF': 0.012544491170372497,
  'CLF': 0.000458417

In [159]:
converted_rates['rates'].keys()

dict_keys(['AED', 'AFN', 'ALL', 'AMD', 'ANG', 'AOA', 'ARS', 'AUD', 'AWG', 'AZN', 'BAM', 'BBD', 'BDT', 'BGN', 'BHD', 'BIF', 'BMD', 'BND', 'BOB', 'BRL', 'BSD', 'BTC', 'BTN', 'BWP', 'BYN', 'BYR', 'BZD', 'CAD', 'CDF', 'CHF', 'CLF', 'CLP', 'CNY', 'CNH', 'COP', 'CRC', 'CUC', 'CUP', 'CVE', 'CZK', 'DJF', 'DKK', 'DOP', 'DZD', 'EGP', 'ERN', 'ETB', 'EUR', 'FJD', 'FKP', 'GBP', 'GEL', 'GGP', 'GHS', 'GIP', 'GMD', 'GNF', 'GTQ', 'GYD', 'HKD', 'HNL', 'HRK', 'HTG', 'HUF', 'IDR', 'ILS', 'IMP', 'INR', 'IQD', 'IRR', 'ISK', 'JEP', 'JMD', 'JOD', 'JPY', 'KES', 'KGS', 'KHR', 'KMF', 'KPW', 'KRW', 'KWD', 'KYD', 'KZT', 'LAK', 'LBP', 'LKR', 'LRD', 'LSL', 'LTL', 'LVL', 'LYD', 'MAD', 'MDL', 'MGA', 'MKD', 'MMK', 'MNT', 'MOP', 'MRU', 'MUR', 'MVR', 'MWK', 'MXN', 'MYR', 'MZN', 'NAD', 'NGN', 'NIO', 'NOK', 'NPR', 'NZD', 'OMR', 'PAB', 'PEN', 'PGK', 'PHP', 'PKR', 'PLN', 'PYG', 'QAR', 'RON', 'RSD', 'RUB', 'RWF', 'SAR', 'SBD', 'SCR', 'SDG', 'SEK', 'SGD', 'SHP', 'SLE', 'SLL', 'SOS', 'SRD', 'STD', 'SVC', 'SYP', 'SZL', 'THB', 'T

In [153]:
# Example DataFrame
data = {
    'Country': ['United States'],
    'Population': [100],
    'Currency': ['USD']
}
df = pd.DataFrame(data)

# Example usage
currency_converter = Currency()
result = currency_converter.one_cent_of_every_person(df, 'AWG')
print(f"Total in MXN accumulated: {result}")

Total in MXN accumulated: 1.8000007397585058
